In [4]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp
import warnings
warnings.simplefilter("ignore")
from pprint import pprint
from tqdm._tqdm_notebook import tqdm_notebook

tqdm_notebook.pandas()

In [6]:
DATA_PATH = 'data'

In [3]:
with open(os.path.join(DATA_PATH, 'catalogue.json'), 'r') as f:
    catalogue = json.load(f)
    
catalogue = {int(k): v for k, v in catalogue.items()}

In [9]:
len(catalogue)

10200

In [22]:
pprint(catalogue[100])

{'attributes': [18441,
                16300,
                16580,
                18770,
                18771,
                18643,
                396,
                18772,
                3771,
                18773,
                910,
                18774,
                16364,
                3277],
 'availability': ['purchase', 'rent'],
 'duration': 80,
 'feature_1': 6064738.740195342,
 'feature_2': 0.752750538,
 'feature_3': 4,
 'feature_4': 0.9537104605,
 'feature_5': 0.0,
 'type': 'movie'}


 - `attributes` — мешок атрибутов
 - `availability` — доступность (может содержать значения `purchase`, `rent` и `subscription`)
 - `duration` — длительность в минутах, округлённая до десятков (продолжительность серии для сериалов и многосерийных фильмов)
 - `feature_1..5` — пять анонимизированных вещественных и порядковых признаков
 - `type` — принимает значения `movie`, `multipart_movie` или `series`

---

`test_users.json` содержит список пользователей, для которых необходимо построить предсказание

In [4]:
with open(os.path.join(DATA_PATH, 'test_users.json'), 'r') as f:
    test_users = set(json.load(f)['users'])

---

`transactions.csv` — список всех транзакций за определённый период времени

In [90]:
transactions = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'consumption_mode': 'category',
        'ts': np.float64,
        'watched_time': np.uint64,
        'device_type': np.uint8,
        'device_manufacturer': np.uint8
    }
)

In [91]:
transactions.head(3)

,element_uid,user_uid,consumption_mode,ts,watched_time,device_type,device_manufacturer
0,3336,5177,S,4.430518e+07,4282,0,50
1,481,593316,S,4.430518e+07,2989,0,11
2,4128,262355,S,4.430518e+07,833,0,50


 - `element_uid` — идентификатор элемента
 - `user_uid` — идентификатор пользователя
 - `consumption_mode` — тип потребления (`P` — покупка, `R` — аренда, `S` — просмотр по подписке)
 - `ts` — время совершения транзакции или начала просмотра в случае просмотра по подписке
 - `watched_time` — число просмотренных по транзакции секунд
 - `device_type` — анонимизированный тип устройства, с которого была совершена транзакция или начат просмотр
 - `device_manufacturer` — анонимизированный производитель устройства, с которого была совершена транзакция или начат просмотр

---

`ratings.csv` содержит информацию о поставленных пользователями оценках

In [7]:
ratings = pd.read_csv(
    os.path.join(DATA_PATH, 'ratings.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64,
        'rating': np.uint8
    }
)

 - `rating` — поставленный пользователем рейтинг (от `0` до `10`)

In [18]:
ratings.head(3)

,user_uid,element_uid,rating,ts
0,571252,1364,10,4.430517e+07
1,63140,3037,10,4.430514e+07
2,443817,4363,8,4.430514e+07


---

`bookmarks.csv` содержит информацию об элементах, добавленных пользователями в список «Избранное»

In [7]:
bookmarks = pd.read_csv(
    os.path.join(DATA_PATH, 'bookmarks.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64
    }
)

In [25]:
bookmarks.head(3)

,user_uid,element_uid,ts
0,301135,7185,4.430516e+07
1,301135,4083,4.430516e+07
2,301135,10158,4.430516e+07


## Решение

Для начала построим список элементов, которые тестовые пользователи уже купили или посмотрели по подписке: они не смогут купить их второй раз, а просмотр по подписке второй раз маловероятен, поэтому мы захотим отфильтровать такие элементы из финального ответа.

Точно так же можно поступить и с рейтингами и добавлениями в избранное, если это будет казаться правильным.

In [8]:
from collections import defaultdict

filtered_elements = defaultdict(set)

for user_uid, element_uid in tqdm_notebook(transactions.loc[:, ['user_uid', 'element_uid']].values):
    if user_uid not in test_users:
        continue
    filtered_elements[user_uid].add(element_uid)

---

Для примера мы воспользуемся методом K ближайших соседей, реализованным в библиотеке `implicit`. В качестве данных используем только информацию о рейтингах.

Необходимо построить разреженную матрицу, где строкам будут соответствовать элементы, столбцам — пользователи, а на пересечении пользователя и элемента будет находиться количественная оценка степени их взаимодействия, если таковое имело место.

Не забудем добавить `1` к рейтингу, чтобы избежать деления на ноль во время вычисления `tf-idf`.

In [9]:
ratings['user_uid'] = ratings['user_uid'].astype('category')
ratings['element_uid'] = ratings['element_uid'].astype('category')

ratings_matrix = sp.coo_matrix(
    (ratings['rating'].astype(np.float32) + 1,
        (
            ratings['element_uid'].cat.codes.copy(),
            ratings['user_uid'].cat.codes.copy()
        )
    )
)

ratings_matrix = ratings_matrix.tocsr()

In [10]:
ratings_matrix

<7519x104563 sparse matrix of type '<class 'numpy.float32'>'
	with 438790 stored elements in Compressed Sparse Row format>

In [11]:
sparsity = ratings_matrix.nnz / (ratings_matrix.shape[0] * ratings_matrix.shape[1])
print('Sparsity: %.6f' % sparsity)

Sparsity: 0.000558


Обучить модель крайне просто.

In [12]:
from implicit.nearest_neighbours import TFIDFRecommender

model = TFIDFRecommender()
model.fit(ratings_matrix)

100%|██████████| 7519/7519 [00:00<00:00, 86002.57it/s]


---

In [13]:
ratings_matrix_T = ratings_matrix.T.tocsr()

Отображения из оригинальной категории во внутреннюю пригодится нам в дальнейшем.

In [14]:
user_uid_to_cat = dict(zip(
    ratings['user_uid'].cat.categories,
    range(len(ratings['user_uid'].cat.categories))
))

In [15]:
element_uid_to_cat = dict(zip(
    ratings['element_uid'].cat.categories,
    range(len(ratings['element_uid'].cat.categories))
))

In [16]:
filtered_elements_cat = {k: [element_uid_to_cat.get(x, None) for x in v] for k, v in filtered_elements.items()}

---

В метод `model.recommend` мы передаём идентификатор пользователя, который получаем обратным преобразованием из категории, транспонированную матрицу взаимодействий, число необходимых рекомендаций и список элементов, которые мы договорились фильтровать из ответа.

Возвращает метод список пар (`element_cat`, `score`), отсортированный по вторым элементам. Из него необходимо достать все первые элементы пар и из категории преобразовать их к `element_uid`.

**Важно:** Не все тестовые пользователи есть в `ratings.csv` и не все из них есть в `transactions.csv`. Используя только один источник данных мы не можем построить полное предсказание. Такой ответ с неполным числом пользователей бдет принят системой, но при вычислении средней метрики метрика для отсутствующих пользователей будет принята равной нулю.

In [17]:
result = {}

for user_uid in tqdm.tqdm(test_users):
    # transform user_uid to model's internal user category
    try:
        user_cat = user_uid_to_cat[user_uid]
    except LookupError:
        continue
    
    # perform inference
    recs = model.recommend(
        user_cat,
        ratings_matrix_T,
        N=20,
        filter_already_liked_items=True,
        filter_items=filtered_elements_cat.get(user_uid, set())
    )
    
    # drop scores and transform model's internal elelemnt category to element_uid for every prediction
    # also convert np.uint64 to int so it could be json serialized later
    result[user_uid] = [int(ratings['element_uid'].cat.categories[i]) for i, _ in recs]

100%|██████████| 50000/50000 [00:02<00:00, 19519.98it/s]


In [18]:
len(result)

13251

Используя только информацию о рейтингах мы смогли построить предсказание для `13251` из `50000` тестовых пользователей. Ровно в таком виде ответы и стоит сохранить для отправки.

In [19]:
with open('answer.json', 'w') as f:
    json.dump(result, f)

## Отсюда мое

In [19]:
ratings = pd.read_csv(
    os.path.join(DATA_PATH, 'ratings.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64,
        'rating': np.uint8
    }
)

In [144]:
def cv_split(n_splits, data_train):
    """
    Разделяем данные на трейн юзеров и тест юзеров по уникальным пользователям
    :n_splits: кол-во фолдов
    :data_train: данные для обучения
    """
    unique_users = np.unique(data_train['user_uid'])
    split_data = np.array_split(unique_users, n_splits)
    
    for fold in range(n_splits):
        train_users, test_users = np.hstack([x for i,x in enumerate(split_data) if i != fold]), split_data[fold]
        yield train_users, test_users

In [137]:
def get_dfs(data_train, count, *cv_fold, data=transactions):
    """
    Оставляем в тесте и трейне только тех у кого более или равно count кол-во фильмов,
    выдаем данные для обучения и оценки
    :data_train: данные для обучения
    :count: кол-во фильмов, для трейна и предикта
    :data: данные для выбора только тех юзеров у которых есть определенное кол-во фильмов
    """
    data = data[data.groupby('user_uid')['element_uid'].transform('size') >= count]
    unique_users = np.unique(data['user_uid'])
    
    train_users, test_users = cv_fold[0], cv_fold[1]
    train_users = np.intersect1d(train_users, unique_users)
    test_users = np.intersect1d(test_users, unique_users)
    train_df = data_train[data_train['user_uid'].isin(train_users)]
    test_df = data_train[data_train['user_uid'].isin(test_users)]
    
    return train_df, test_df

In [ ]:
train_df['user_uid'] = train_df['user_uid'].astype('category')
train_df['element_uid'] = train_df['element_uid'].astype('category')

ratings_matrix = sp.coo_matrix(
    (train_df['rating'].astype(np.float32) + 1,
        (
            train_df['element_uid'].cat.codes.copy(),
            train_df['user_uid'].cat.codes.copy()
        )
    )
)

user_uid_to_cat = dict(zip(
    train_df['user_uid'].cat.categories,
    range(len(train_df['user_uid'].cat.categories))
))

ratings_matrix = ratings_matrix.tocsr()
ratings_matrix_T = ratings_matrix.T.tocsr()

In [173]:
def model_fit(ratings_matrix):
    """ 
    Обучение модели на разреженной матрице
    """
    model = TFIDFRecommender()
    model.fit(ratings_matrix)
    
    return model

In [169]:
counter = cv_split(5, ratings)
it = next(counter)
train_df, test_df = get_dfs(ratings, 20, *it)
model, ratings_matrix_T, user_uid_to_cat = model_fit(train_df)

In [192]:
result_test = {}

for user_uid in tqdm.tqdm(test_df.user_uid.values):
    try:
        user_cat = user_uid_to_cat[user_uid]
    except LookupError:
        continue

    recs = model.recommend(
        user_cat,
        ratings_matrix_T,
        N=20,)
#         filter_already_liked_items=True,
#         filter_items=filtered_elements_cat.get(user_uid, set())
#     )

    result_test[user_uid] = [int(train_df['element_uid'].cat.categories[i]) for i, _ in recs]

100%|██████████| 60794/60794 [00:00<00:00, 659249.60it/s]


In [ ]:
transactions_rel = transactions[transactions.groupby('user_uid')['element_uid'].transform('size') >= 20]

In [24]:
len(user_uid_to_cat_train)

73194

In [23]:
train_ratings['user_uid'] = train_ratings['user_uid'].astype('category')
train_ratings['element_uid'] = train_ratings['element_uid'].astype('category')

user_uid_to_cat_train = dict(zip(
    train_ratings['user_uid'].cat.categories,
    range(len(train_ratings['user_uid'].cat.categories))
))
ratings_matrix_train = sp.coo_matrix(
    (train_ratings['rating'].astype(np.float32) + 1,
        (
            train_ratings['element_uid'].cat.codes.copy(),
            train_ratings['user_uid'].cat.codes.copy()
        )
    )
)

ratings_matrix_train = ratings_matrix_train.tocsr()
ratings_matrix_train_T = ratings_matrix_train.T.tocsr()
ratings_matrix_train

<7157x73194 sparse matrix of type '<class 'numpy.float32'>'
	with 307769 stored elements in Compressed Sparse Row format>

In [25]:
from implicit.nearest_neighbours import TFIDFRecommender

model = TFIDFRecommender()
model.fit(ratings_matrix_train)

100%|██████████| 7157/7157 [00:00<00:00, 61611.44it/s]


In [38]:
# Скопировал код с https://habr.com/ru/company/okko/blog/439180/ хз правильно ли отредактировал с CPython
def average_precision(data_true, data_predicted, k) -> float:

    if not data_true:
        raise ValueError('data_true is empty')

    average_precision_sum = 0.0

    for key, items_true in data_true.items():
        items_predicted = data_predicted.get(key, [])

        n_items_true = len(items_true)
        n_items_predicted = min(len(items_predicted), k)

        if n_items_true == 0 or n_items_predicted == 0:
            continue

        n_correct_items = 0
        precision = 0.0

        for item_idx in range(n_items_predicted):
            if items_predicted[item_idx] in items_true:
                n_correct_items += 1
                precision += n_correct_items / (item_idx + 1)

        average_precision_sum += precision / min(n_items_true, k)

    return average_precision_sum / len(data_true)

In [39]:
def metric(true_data, predicted_data, k=20):
    true_data_set = {k: set(v) for k, v in true_data.items()}

    return average_precision(true_data_set, predicted_data, k=k)

### Для валидации оставляю из трейна только юзеров у которых более или равно 20 фильмов. Для релевантности, отфильтрую по времени просмотра, чем больше смотрел, тем более релевантен фильм

In [25]:
transactions.shape

(9643012, 7)

Оставляю только тех у кого более 20 фильмов

In [26]:
transactions_rel = transactions[transactions.groupby('user_uid')['element_uid'].transform('size') >= 20]

Выбираю фильмы в порядке релевантности

In [27]:
transactions_rel.sort_values(by=['user_uid', 'watched_time'], ascending=False, inplace=True)

In [28]:
transactions_rel20 = transactions_rel.groupby('user_uid').head(20)

In [29]:
transactions_rel.head(3)

,element_uid,user_uid,consumption_mode,ts,watched_time,device_type,device_manufacturer
7633763,2760,593489,S,4.231365e+07,9057,0,50
6833184,747,593489,S,4.257764e+07,7678,0,50
212525,8286,593489,S,4.425567e+07,7560,0,50


Создаю словарь с верными данными по релевантности

In [29]:
true_data = transactions_rel20.groupby('user_uid').element_uid.progress_apply(list).to_dict()

In [30]:
len(true_data)

141339

Оставляю только тех кто есть в ratings.user_uid

In [31]:
filter_user_id = np.intersect1d(np.unique(train_ratings.user_uid),np.unique(transactions_rel20.user_uid))
filtered_true_data = {k:v for k,v in true_data.items() if k in filter_user_id}

In [32]:
len(filtered_true_data)

32505

Предсказываем для transactions_rel

In [33]:
transactions_rel['user_uid'].shape

(6796629,)

In [35]:
result_train = {}

for user_uid in tqdm_notebook(transactions_rel['user_uid']):
    # transform user_uid to model's internal user category
    try:
        user_cat = user_uid_to_cat_train[user_uid]
    except LookupError:
        continue
    
    # perform inference
    recs = model.recommend(
        user_cat,
        ratings_matrix_train_T,
        N=20,
#         filter_already_liked_items=True,
#         filter_items=filtered_elements_cat.get(user_uid, set())
     )
    
    result_train[user_uid] = [int(train_ratings['element_uid'].cat.categories[i]) for i, _ in recs]

In [36]:
len(result_train)

32505

In [40]:
metric(filtered_true_data, result_train, k=20)# модель дает скор 0.00468558 на паблике

0.02567712640647301

### На отложенной выборке

In [41]:
filter_user_id = np.intersect1d(split_data[1],np.unique(transactions_rel20.user_uid))
filtered_true_data = {k:v for k,v in true_data.items() if k in filter_user_id}

In [42]:
len(filtered_true_data)

14039

In [43]:
test_ratings['user_uid'] = test_ratings['user_uid'].astype('category')
test_ratings['element_uid'] = test_ratings['element_uid'].astype('category')

user_uid_to_cat_test = dict(zip(
    test_ratings['user_uid'].cat.categories,
    range(len(test_ratings['user_uid'].cat.categories))
))
ratings_matrix_test = sp.coo_matrix(
    (test_ratings['rating'].astype(np.float32) + 1,
        (
            test_ratings['element_uid'].cat.codes.copy(),
            test_ratings['user_uid'].cat.codes.copy()
        )
    )
)

ratings_matrix_test = ratings_matrix_test.tocsr()
ratings_matrix_test_T = ratings_matrix_test.T.tocsr()
ratings_matrix_test

<5909x31369 sparse matrix of type '<class 'numpy.float32'>'
	with 131021 stored elements in Compressed Sparse Row format>

In [72]:
len(user_uid_to_cat_test)

31369

In [71]:
len(user_uid_to_cat_train)

73194

In [67]:
train_ratings['element_uid'].head()

1    3037
3    1364
4    3578
6    8273
7    5368
Name: element_uid, dtype: category
Categories (7157, uint64): [3, 4, 6, 7, ..., 10194, 10196, 10197, 10199]

In [68]:
test_ratings['element_uid'].head()

0     1364
2     4363
5     1918
14    3916
18    2650
Name: element_uid, dtype: category
Categories (5909, uint64): [3, 4, 6, 7, ..., 10185, 10186, 10187, 10199]

In [73]:
ratings_matrix_test_T

<31369x5909 sparse matrix of type '<class 'numpy.float32'>'
	with 131021 stored elements in Compressed Sparse Row format>

In [74]:
    recs = model.recommend(
        user_cat,
        ratings_matrix_test_T,
        N=20,
#         filter_already_liked_items=True,
#         filter_items=filtered_elements_cat.get(user_uid, set())
     )

In [78]:
train_ratings['element_uid'].cat.categories[7079]

10086

In [75]:
recs

[(7079, 8.344319094340614),
 (4404, 2.0382054354515238),
 (5021, 2.0382054354515238),
 (5055, 2.0382054354515238),
 (4603, 2.0382054354515238),
 (5030, 2.0382054354515238),
 (6147, 2.0382054354515238),
 (5397, 2.0382054354515238),
 (5570, 2.0382054354515238),
 (4653, 2.0382054354515238),
 (6342, 2.0382054354515238),
 (6062, 2.0382054354515238),
 (6444, 2.0382054354515238),
 (6411, 2.0382054354515238),
 (6654, 2.0382054354515238),
 (5477, 2.0382054354515238),
 (6507, 2.0382054354515238),
 (6236, 2.0382054354515238),
 (4693, 2.0382054354515238),
 (7155, 0.5674585143832721)]

In [79]:
result_test = {}

for user_uid in tqdm_notebook(transactions_rel['user_uid']):
    # transform user_uid to model's internal user category
    try:
        user_cat = user_uid_to_cat_test[user_uid]
    except LookupError:
        continue
    
    # perform inference
    recs = model.recommend(
        user_cat,
        ratings_matrix_test_T,
        N=20,
#         filter_already_liked_items=True,
#         filter_items=filtered_elements_cat.get(user_uid, set())
     )
    
    result_test[user_uid] = [int(train_ratings['element_uid'].cat.categories[i]) for i, _ in recs]

In [87]:
len(result_test)

14039

In [94]:
metric(filtered_true_data, result_test, k=30)# модель дает скор 0.00468558 на паблике

0.00029796284806914717